In [ ]:
!conda install -c conda-forge gdcm -y

In [ ]:
# library import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, glob, pickle, gc, copy, sys, multiprocessing
from joblib import Parallel, delayed

import warnings
import cv2, pydicom
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.optim.lr_scheduler import _LRScheduler

sys.path.append('../input/timm-efficientnet/pytorch-image-models-master/')
import timm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

# params
col_index = 'SOPInstanceUID'
col_groupby = 'StudyInstanceUID'
BATCH_SIZE = 64
col_targets = [
    'negative_exam_for_pe',
    'indeterminate',
    'chronic_pe',
    'acute_and_chronic_pe',
    'central_pe',
    'leftsided_pe',
    'rightsided_pe',
    'rv_lv_ratio_gte_1',
    'rv_lv_ratio_lt_1',
    'pe_present_on_image',
]
col_targets_display = [
    'Negative Exam for PE',
    'Indeterminate',
    'Chronic PE',
    'Acute and Chronic PE',
    'Central PE',
    'Left-sided PE',
    'Right-sided PE',
    'RV/LV Ratio: > or = 1',
    'RV/LV Ratio: < 1',
    'PE Present on Image',
]
num_features1_1 = 1280

# data loading
df_valid = pd.read_csv("../input/rsna2020-demo/df_valid.csv")
df_valid_exam = pd.read_csv("../input/rsna2020-demo/df_valid_exam.csv")
df_valid_exam['3class'] = 1
df_valid_exam['3class'][df_valid_exam['negative_exam_for_pe']==1] = 0
df_valid_exam['3class'][df_valid_exam['indeterminate']==1] = 2

# model loading
def get_images(idx):
    exam = df_valid_exam[col_groupby][idx]
    start_index = df_valid_exam['start_index'][idx]
    end_index = start_index + df_valid_exam['num_series'][idx]
    df_tmp = df_valid.iloc[start_index:end_index].reset_index(drop=True)
    RescaleSlope = df_valid_exam['RescaleSlope'][idx]
    RescaleIntercept = df_valid_exam['RescaleIntercept'][idx]
    # load dicoms
    images_exam = []
    z_pos = []
    for i in range(len(df_tmp)):
        tmp_path = df_tmp['path'][i]
#         print(tmp_path)
        tmp_dcm = pydicom.dcmread(tmp_path)
        tmp_npy = np.asarray(tmp_dcm.pixel_array)
        images_exam.append(tmp_npy)

    # process images
    images_exam = np.array(images_exam)
    images_exam_processed = (images_exam.astype(np.float32) * RescaleSlope + RescaleIntercept)/1000
    images_exam_processed = images_exam_processed.reshape([-1, 1, 512, 512]).astype(np.float16)

    return images_exam_processed

class nnWindow(nn.Module):
    def __init__(self):
        super(nnWindow, self).__init__()
        wso = np.array(((40,80),(80,200),(40,400)))/1000
        conv_ = nn.Conv2d(1,3, kernel_size=(1, 1))
        conv_.weight.data.copy_(torch.tensor([[[[1./wso[0][1]]]],[[[1./wso[1][1]]]],[[[1./wso[2][1]]]]]))
        conv_.bias.data.copy_(torch.tensor([0.5 - wso[0][0]/wso[0][1],
                                            0.5 - wso[1][0]/wso[1][1],
                                            0.5 -wso[2][0]/wso[2][1]]))
        self.window = nn.Sequential(
            conv_,
            nn.Sigmoid(),
            nn.InstanceNorm2d(3)
        )
    def forward(self, input1):
        return self.window(input1)
        
        
class CNN_2D(nn.Module):
    def __init__(self, num_classes=10, base_model='tf_efficientnet_b0_ns'):
        super(CNN_2D, self).__init__()

        self.num_classes = num_classes
        self.mode = 'train'
        self.window = nnWindow()
#         self.base_model = pretrainedmodels.__dict__['resnet18'](num_classes=1000, pretrained='imagenet')
        self.base_model = timm.create_model(base_model, pretrained=False, num_classes=10).to(device, non_blocking=True)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
#         self.last_linear = nn.Linear(512, num_classes+1)
        self.last_linear = nn.Linear(self.base_model.num_features, num_classes)

    def forward(self, input1):
        bs, ch, h, w = input1.size()
        x = self.window(input1)
        x = self.base_model.forward_features(x) #; print('layer conv1 ',x.size()) # [8, 64, 112, 112]
        feature = self.avgpool(x).view(bs, -1)
        y = self.last_linear(feature)

        return y

    def feature(self, input1):
        bs, ch, h, w = input1.size()
        x = self.window(input1)
        x = self.base_model.forward_features(x) #; print('layer conv1 ',x.size()) # [8, 64, 112, 112]
        feature = self.avgpool(x).view(bs, -1)
        y = self.last_linear(feature)

        return y, feature
    

class SEModule(nn.Module):

    def __init__(self, channels, reduction):
        super(SEModule, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc1 = nn.Conv1d(channels, channels // reduction, kernel_size=1,
                             padding=0)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv1d(channels // reduction, channels, kernel_size=1,
                             padding=0)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        module_input = x
        x = self.avg_pool(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return module_input * x
    
class CNN_1D(nn.Module):

    def __init__(self, num_classes=400, input_ch=1280, verbose=False):

        super(CNN_1D, self).__init__()
        pool = 4
        drop = 0.1
        self.verbose = verbose
        self.layer1 = nn.Sequential(
                nn.Conv1d(input_ch//pool, 64, kernel_size=7, stride=1, padding=3, bias=False),
                nn.BatchNorm1d(64),
                nn.ReLU(inplace=True),
                SEModule(64, 16),
#                 nn.Dropout(drop),
        )
        self.fpool = nn.MaxPool1d(kernel_size=pool, stride=pool, padding=0)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
#         self.upsample = nn.Upsample(scale_factor=2, mode='bilinear')
        self.layer2 = nn.Sequential(
                nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1, bias=False),
                nn.BatchNorm1d(128),
                nn.ReLU(inplace=True),
                SEModule(128, 16),
#                 nn.Dropout(drop),
        )
        self.layer3 = nn.Sequential(
                nn.Conv1d(128, 256, kernel_size=3, stride=1, padding=1, bias=False),
                nn.BatchNorm1d(256),
                nn.ReLU(inplace=True),
                SEModule(256, 16),
#                 nn.Dropout(drop),
        )
        self.layer4 = nn.Sequential(
                nn.Conv1d(256, 512, kernel_size=3, stride=1, padding=1, bias=False),
                nn.BatchNorm1d(512),
                nn.ReLU(inplace=True),
                SEModule(512, 16),
#                 nn.Dropout(drop),
        )
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc2 = nn.Conv1d(
            input_ch//pool+64+128+256+512, 
            2, kernel_size=1)
#         self.fc = nn.Linear(512, 9)
        self.fc = nn.Sequential(
                nn.Linear(512, 512),
                nn.ReLU(inplace=True),
                nn.Dropout(0.5),
                nn.Linear(512, 512),
                nn.ReLU(inplace=True),
                nn.Dropout(0.5),
                nn.Linear(512, 9),
        )

    def forward(self, x_input):
        bs, ch, d = x_input.size()
        x0 = torch.transpose(x_input, 1, 2)
        x0 = self.fpool(x0)
        x0 = torch.transpose(x0, 1, 2)
        x1 = self.layer1(x0)
        x1 = self.maxpool(x1)

        x2 = self.layer2(x1)
        x2 = self.maxpool(x2)
        x3 = self.layer3(x2)
        x3 = self.maxpool(x3)
        x4 = self.layer4(x3)
        
#         tmp = F.adaptive_avg_pool1d(x1, d)
#         print(tmp.shape)
#         tmp = F.adaptive_avg_pool1d(x2, d)
#         print(tmp.shape)
        x5 = torch.cat([
            x0,
            F.adaptive_avg_pool1d(x1, d), 
            F.adaptive_avg_pool1d(x2, d), 
            F.adaptive_avg_pool1d(x3, d), 
            F.adaptive_avg_pool1d(x4, d), 
        ], axis=1)
        y2 = self.fc2(x5)
        
        b, ch, d = x_input.size()
#         x1 = self.fc(x)
#         x1 = x1.view(b, -1, 1)
            
        y = self.avgpool(x4)
        y = y.view(b, -1)
        y = self.fc(y)
        return y, y2
    
cnn_2d = CNN_2D().to(device, non_blocking=True)
cnn_2d.load_state_dict(torch.load("../input/rsna2020-pretrained-weights/b0_stage1/weight_epoch_16_fold1.pth"))
# cnn_2d.load_state_dict(torch.load("../input/rsna2020-1/201022_6_CNN_b0_1loss_512_fp16_bs80_lr1e3 (2)/201022_6_CNN_b0_1loss_512_fp16_bs80_lr1e3/weight_epoch_16_fold1.pth"))
cnn_1d = CNN_1D().to(device, non_blocking=True)
cnn_1d.load_state_dict(torch.load("../input/rsna2020-pretrained-weights/b0_stage2/1dcnn_weight_best_fold1.pth"))
# cnn_1d.load_state_dict(torch.load("../input/rsna2020-1/201025_3_2ndNNs_features_pool_flip_new/201025_3_2ndNNs_features_pool_flip_new/cnn_weight_best_fold1.pth"))
lastfunc = nn.Sigmoid().to(device, non_blocking=True)
cnn_2d.eval()
cnn_1d.eval()

# function definition
def batch_padding(batch):
    bs, ch, d = batch.shape
    d_new = int(np.ceil(d/64)*64)
#     d_new = int(np.ceil(1083/64)*64)
    batch_new = torch.from_numpy(np.zeros([bs, ch, d_new], np.float32)).to(device, non_blocking=True)
    batch_new[:, :, :d] = batch
    return batch_new

def window(img, WL=50, WW=350):
    upper, lower = WL+WW//2, WL-WW//2
    X = np.clip(img.copy(), lower, upper)
    X = X - np.min(X)
    X = X / np.max(X)
    X = (X*255.0).astype('uint8')
    return X

def get_demo(idx):
    # get exam data
    exam = df_valid_exam[col_groupby][idx]
    print("index: {}".format(idx))
    print("{}: {}".format(col_groupby, exam))
    start_index = df_valid_exam['start_index'][idx]
    end_index = start_index + df_valid_exam['num_series'][idx]
    df_tmp = df_valid.iloc[start_index:end_index].reset_index(drop=True)
    images = get_images(idx)
    
    
    # model prediction
    BATCH_SIZE = 64
    batchs = torch.from_numpy(images).to(device, non_blocking=True)
    num_batches = int(np.ceil(images.shape[0]/BATCH_SIZE))
    features = []
    output0s = []
    for batch_index in range(num_batches):
        with torch.no_grad():
            with torch.cuda.amp.autocast():
                batch = batchs[batch_index*BATCH_SIZE:(batch_index+1)*BATCH_SIZE].to(device, non_blocking=True)
                output0, feature = cnn_2d.feature(batch)
                output0 = lastfunc(output0)
        features.append(feature)
        output0s.append(output0)
    features = torch.cat(features, axis=0) # bs=d, ch
    features = torch.transpose(features, 0,1).reshape([1, num_features1_1, -1])
    features = batch_padding(features)
    output0s = torch.cat(output0s, axis=0).data.cpu().numpy()
    with torch.no_grad():
        output1, output2 = cnn_1d(features)
        output2 = output2[:,-1:]
        output1 = lastfunc(output1).data.cpu().numpy()[0]

        output2 = lastfunc(output2)[:,:,:len(images)].data.cpu().numpy()[0,0]
    
    # print exam-level pred
    print('Exam-Level Labels         Prediction    True')
    for i, item in enumerate(col_targets[:-1]):
        col_name = "{}                          ".format(col_targets_display[i])[:25]
        print("{} {:.6f}      {}".format(col_name, output1[i], df_valid_exam[col_targets[i]][idx]))
    
    # show exam-level pred
    for i in range(9):
        plt.figure(figsize=(20,5))
        plt.subplot(1,3,1)
        plt.scatter(np.arange(len(df_tmp)), output0s[:,i], alpha=1, s=3, label="Pred {}".format(col_targets[i]))
        plt.scatter(output0s[:,i].argmax(), output0s[:,i].max(), alpha=1, s=50, label='Max')
        plt.scatter(output0s[:,i].argmin(), output0s[:,i].min(), alpha=1, s=50, label='Min')
        plt.ylim(-0.1,1.1)
        plt.legend(fontsize=12)
        plt.grid()
        plt.ylabel('score', fontsize=12)
        plt.xlabel('series index', fontsize=12)
        plt.title("Label:                  {}\nExam-lebel Pred: {:.6f}\nTrue:                    {}".format(
            col_targets_display[i], output1[i], df_valid_exam[col_targets[i]][idx]), fontsize=18, loc='left',
                 color=('tab:red' if df_valid_exam[col_targets[i]][idx]==1 else 'tab:blue')
                 )
        plt.subplot(1,3,2)
        image = images[output0s[:,i].argmax(),0].astype(np.float32)
        image = window(image*1000, 100, 700)
        plt.imshow(image, cmap='gray')
        plt.title("Max\nSeries index:        {}\nImage-level Pred: {:.6f}".format(output0s[:,i].argmax(), output0s[:,i].max()), fontsize=18, loc='left')
        plt.subplot(1,3,3)
        image = images[output0s[:,i].argmin(),0].astype(np.float32)
        image = window(image*1000, 100, 700)
        plt.imshow(image, cmap='gray')
        plt.title("Min\nSeries index:        {}\nImage-level Pred: {:.6f}".format(output0s[:,i].argmin(), output0s[:,i].min()), fontsize=18, loc='left')
        plt.show()
    
    # show image-level pred
    plt.figure(figsize=(20,5))
    plt.subplot(1,3,1)
    plt.scatter(np.arange(len(df_tmp)), output2, alpha=1, s=3, label="Pred PE present", color='#1f77b4')
    plt.scatter(output2.argmax(), output2.max(), alpha=1, s=50, label='Max', color='#ff7f0e')
    plt.scatter(output2.argmin(), output2.min(), alpha=1, s=50, label='Min', color='#2ca02c')
    plt.scatter(np.arange(len(df_tmp)), df_tmp[col_targets[-1]], alpha=1, s=10, label="True PE present", color='tab:red', zorder=-1)
    plt.ylim(-0.1,1.1)
    plt.legend(fontsize=12)
    plt.grid()
    plt.ylabel('score', fontsize=12)
    plt.xlabel('series index', fontsize=12)
    plt.title("Label: {}\n".format(col_targets_display[-1]), fontsize=18, loc='left')
    plt.subplot(1,3,2)
    image = images[output2.argmax(),0].astype(np.float32)
    image = window(image*1000, 100, 700)
    plt.imshow(image, cmap='gray')
    plt.title("Max\nSeries index:        {}\nImage-level Pred: {:.6f}".format(output2.argmax(), output2.max()), fontsize=18, loc='left')
    plt.subplot(1,3,3)
    image = images[output2.argmin(),0].astype(np.float32)
    image = window(image*1000, 100, 700)
    plt.imshow(image, cmap='gray')
    plt.title("Min\nSeries index:        {}\nImage-level Pred: {:.6f}".format(output2.argmin(), output2.min()), fontsize=18, loc='left')
    plt.show()

Run the cell below.  
It shows a visualization of a model prediction of 1 CT scan selected at random.

In [ ]:
idx = np.random.randint(len(df_valid_exam))# sampling at random
# idx = np.random.choice(df_valid_exam[df_valid_exam['3class']==1].index.values) # sampling from PE-positive scan
# idx = np.choice(df_valid_exam[df_valid_exam['3class']==0]) # sampling from PE-negative scan
idx = 878 # please comment out this line to sample CT scan at random
get_demo(idx)